# JST

In [3]:
import numpy as np

import pandas as pd
from pandarallel import pandarallel
from reviews.config import processed_data_dir, asum_input_dir, data_dir
from reviews.preprocess import preprocess

pandarallel.initialize(progress_bar=True, nb_workers=8)


INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [3]:
reviews_df = pd.read_json(
    processed_data_dir / "reviews_lemmatization.json.gz", orient="records"
)
reviews_df = reviews_df[:1000]

In [ ]:
tokens = reviews_df['tokens']
func = lambda x:[item for sublist in list(x) for item in sublist]
tokens = [" ".join(func(token)) for token in tokens]
tokens

In [6]:
#tokens = reviews_df["text"].apply(lambda x: preprocess(x, lemmatization=True, sentences=False, return_tokens=False))

In [17]:
docs = list(tokens)
docs = [f"d{i} {x}" for i, x in enumerate(docs)]

docs_file = "\n".join(docs)

with open(data_dir / "jst" / "docs.dat", "w") as f:
    f.write(docs_file)

In [18]:
vocabs = [x for sublist in list(tokens) for x in sublist.split(" ")]

In [19]:
from collections import Counter
vocab_file = "\n".join([f"{k} {v}"  for (k, v) in dict(Counter(vocabs)).items()])

with open(data_dir / "jst" / "wordmap.txt", "w") as f:
    f.write(vocab_file)

In [1]:
from reviews.config import data_dir, bin_dir, out_dir
from reviews.models import JST

alpha = 0.1
beta = 0.001 # [0.001, 0.1, 0]
gamma = 1 #[1, 1]
n_topics = 20
iterations = 1000

model = JST(bin_dir, data_dir / "jst", out_dir / "jst")
model.estimate(alpha, beta, gamma, n_topics, iterations)

C:\Users\mirco\Desktop\amazon-reviews\bin\jst.exe -est -alpha 0.1 -beta 0.001 -gamma 1 -ntopics 20 -niters 1000 -result_dir C:\Users\mirco\Desktop\amazon-reviews\output\jst -data_dir C:\Users\mirco\Desktop\amazon-reviews\data\jst -datasetFile docs.dat -sentiFile C:\Users\mirco\Desktop\amazon-reviews\data\jst\sentiwords.txt


In [45]:
with open(out_dir/ "jst" / "final.phi") as f:
    lines = f.readlines()

    phi = {}
    tmp = ""
    for i,line in enumerate(lines):
        if i % 2 != 0:
            phi[tmp] = [float(x.strip()) for x in line.split(" ") if x.strip() != ""]
        else:
            tmp = line.strip()

phi = pd.DataFrame(phi)
phi
# cambia label in S e topic in T nella forma S0-T0

,Label:0 Topic:0,Label:0 Topic:1,Label:0 Topic:2,Label:0 Topic:3,Label:0 Topic:4,Label:0 Topic:5,Label:0 Topic:6,Label:0 Topic:7,Label:0 Topic:8,Label:0 Topic:9,...,Label:2 Topic:10,Label:2 Topic:11,Label:2 Topic:12,Label:2 Topic:13,Label:2 Topic:14,Label:2 Topic:15,Label:2 Topic:16,Label:2 Topic:17,Label:2 Topic:18,Label:2 Topic:19
0,9.265674e-07,1.741395e-06,1.452956e-06,2.060784e-06,1.911124e-06,1.455070e-06,1.101016e-06,2.015104e-06,1.407950e-06,2.329633e-06,...,2.200773e-06,1.238565e-06,1.794090e-06,1.947853e-06,2.064471e-06,1.509694e-06,2.622018e-06,1.967010e-06,1.843706e-06,7.050268e-07
1,4.632837e-08,8.706975e-08,4.068283e-02,1.030392e-07,9.555621e-08,7.275349e-08,5.505079e-08,1.007552e-07,7.039752e-08,1.164816e-07,...,1.100387e-07,6.192827e-08,8.970449e-08,9.739266e-08,1.032235e-07,7.548472e-08,1.311009e-07,9.835052e-08,9.218532e-08,3.877651e-02
2,9.265674e-07,1.741395e-06,1.452956e-06,2.060784e-06,5.160226e-02,1.455070e-06,1.101016e-06,2.015104e-06,1.407950e-06,2.329633e-06,...,2.200773e-06,1.238565e-06,1.794090e-06,1.947853e-06,2.064471e-06,1.509694e-06,2.622018e-06,1.967010e-06,1.843706e-06,7.050268e-07
3,9.265674e-07,1.741395e-06,3.152928e-01,2.060784e-06,1.911124e-06,1.455070e-06,1.101016e-06,2.015104e-06,1.407950e-06,2.329633e-06,...,2.200773e-06,1.238565e-06,2.834680e-01,1.947853e-06,2.064471e-06,1.509694e-06,2.622018e-06,1.967010e-06,1.843706e-06,7.050268e-07
4,9.265674e-07,1.741395e-06,1.452956e-06,2.060784e-06,1.911124e-06,1.455070e-06,1.101016e-06,2.015104e-06,1.407950e-06,2.329633e-06,...,2.200773e-06,1.238565e-06,1.794090e-06,1.947853e-06,2.064471e-06,1.509694e-06,2.622018e-06,1.967010e-06,1.843706e-06,7.050268e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5575,9.265674e-07,1.741395e-06,1.452956e-06,2.060784e-06,1.911124e-06,1.455070e-06,1.101016e-06,2.015104e-06,1.407950e-06,2.329633e-06,...,2.200773e-06,1.238565e-06,1.794090e-06,1.947853e-06,2.064471e-06,1.509694e-06,2.622018e-06,1.967010e-06,1.843706e-06,7.050268e-07
5576,9.265674e-07,1.741395e-06,1.452956e-06,2.060784e-06,1.911124e-06,1.455070e-06,1.101016e-06,2.015104e-06,1.407950e-06,2.329633e-06,...,2.200773e-06,1.238565e-06,1.794090e-06,1.947853e-06,2.064471e-06,1.509694e-06,2.622018e-06,1.967010e-06,1.843706e-06,7.050268e-07
5577,9.265674e-07,1.741395e-06,1.452956e-06,2.060784e-06,1.911124e-06,1.455070e-06,1.101016e-06,2.015104e-06,1.407950e-06,2.329633e-06,...,2.200773e-06,1.238565e-06,1.794090e-06,1.947853e-06,2.066535e-03,1.509694e-06,2.622018e-06,1.967010e-06,1.843706e-06,7.050268e-07
5578,4.632837e-08,8.706975e-08,7.264778e-08,1.030392e-07,9.555621e-08,7.275349e-08,5.505079e-08,1.007552e-07,7.039752e-08,1.164816e-07,...,1.100387e-07,6.192827e-08,8.970449e-08,9.739266e-08,1.032235e-07,7.548472e-08,1.311009e-07,9.835052e-08,9.218532e-08,3.525134e-08


In [11]:
pi = pd.read_csv(out_dir/ "jst" / "final.pi", sep = " ", header=None)
pi.head()

#rinominare s0 s1 s2, togliere prime due colonne e la quinta colonna

,0,1,2,3,4,5
0,d_0,d0,0.362069,0.396552,0.241379,NaN
1,d_1,d1,0.534884,0.186047,0.279070,NaN
2,d_2,d2,0.066667,0.466667,0.466667,NaN
3,d_3,d3,0.184615,0.384615,0.430769,NaN
4,d_4,d4,0.200000,0.600000,0.200000,NaN


In [32]:
with open(out_dir/ "jst" / "final.theta") as f:
    lines = f.readlines()

    theta = []
    tmp = []
    for i,line in enumerate(lines):
        if i % 4 != 0:
            values = [float(x.strip()) for x in line.split(" ") if x.strip() != ""]
            tmp.extend(values)
        else:
            if tmp != []:
                theta.append(tmp)
                tmp = []

pd.DataFrame(theta, columns=[])
#S0-T0 ..... S0-T19 S1-T0 ...... S2-T19

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,0.050000,0.004545,0.004545,0.004545,0.140909,0.004545,0.004545,0.004545,0.004545,0.004545,...,0.006667,0.140000,0.006667,0.006667,0.006667,0.073333,0.006667,0.006667,0.006667,0.006667
1,0.004167,0.004167,0.004167,0.004167,0.295833,0.004167,0.170833,0.004167,0.004167,0.004167,...,0.007692,0.084615,0.007692,0.392308,0.007692,0.007692,0.007692,0.007692,0.007692,0.007692
2,0.025000,0.025000,0.025000,0.025000,0.275000,0.025000,0.025000,0.025000,0.025000,0.025000,...,0.004545,0.368182,0.004545,0.004545,0.095455,0.140909,0.004545,0.095455,0.004545,0.004545
3,0.007692,0.084615,0.007692,0.007692,0.392308,0.007692,0.084615,0.007692,0.007692,0.007692,...,0.003448,0.486207,0.003448,0.072414,0.003448,0.037931,0.003448,0.003448,0.072414,0.003448
4,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.366667,0.033333,0.033333,0.033333,...,0.033333,0.033333,0.033333,0.366667,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,0.025000,0.025000,0.025000,0.025000,0.025000,0.275000,0.025000,0.025000,0.025000,0.025000,...,0.016667,0.016667,0.016667,0.183333,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667
995,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,...,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000
996,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.275000,...,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000
997,0.358065,0.003226,0.100000,0.003226,0.003226,0.003226,0.003226,0.003226,0.003226,0.293548,...,0.004000,0.404000,0.004000,0.044000,0.004000,0.004000,0.004000,0.004000,0.004000,0.004000
